## [为什么我们需要Pod？](https://time.geekbang.org/column/article/40092)

### 列出当前系统中正在运行的进程的树状结构

```bash
$ pstree -g
systemd(1)─┬─ModemManager(1406)─┬─{ModemManager}(1406)
           │                    └─{ModemManager}(1406)
           ├─NetworkManager(1404)─┬─{NetworkManager}(1404)
           │                      └─{NetworkManager}(1404)
           ├─accounts-daemon(1388)─┬─{accounts-daemon}(1388)
           │                       └─{accounts-daemon}(1388)
           ├─acpid(1440)
           ├─avahi-daemon(29037)───avahi-daemon(29037)
           ├─boltd(2931)─┬─{boltd}(2931)
           │             └─{boltd}(2931)
           ├─colord(3044)─┬─{colord}(3044)
           │              └─{colord}(3044)
           ├─cron(1447)
           ├─cups-browsed(38085)─┬─{cups-browsed}(38085)
           │                     └─{cups-browsed}(38085)
           ├─cupsd(38080)
           ├─dbus-daemon(1390)
           ├─dockerd(2318)─┬─docker-containe(2416)─┬─docker-containe(6900)─┬─pause(6928)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       ├─{docker-containe}(6900)
           │               │                       │                       └─{docker-containe}(6900)
           │               │                       ├─docker-containe(13892)─┬─pause(13946)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        ├─{docker-containe}(13892)
           │               │                       │                        └─{docker-containe}(13892)
           │               │                       ├─docker-containe(14168)─┬─nginx(14194)───nginx(14194)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        ├─{docker-containe}(14168)
           │               │                       │                        └─{docker-containe}(14168)
            ...
```

### 资源调度
* Mesos 中就有一个资源囤积（resource hoarding）的机制，会在所有设置了 Affinity 约束的任务都达到时，才开始对它们统一进行调度。
* Google Omega 论文中，则提出了使用乐观调度处理冲突的方法，即：先不管这些冲突，而是通过精心设计的回滚机制在出现了冲突之后解决问题。
* Kubernetes 中 Pod 是原子调度单位。这就意味着，Kubernetes 项目的调度器，是统一按照 Pod 而非容器的资源需求进行计算的。

### Pod 实现原理

* Pod，其实是一组共享了某些资源的容器。Pod 里的所有容器，共享的是同一个 Network Namespace，并且可以声明共享同一个 Volume。

* 在 Docker 中容器共享 Network 和 Volume。

```bash
$ docker run --net=B --volumes-from=B --name=A image-A
```

* 在这个 Pod 中，Infra 容器永远都是第一个被创建的容器，而其他用户定义的容器，则通过 Join Network Namespace 的方式，与 Infra 容器关联在一起。
![](images/13_container_relationship_in_pod.png)

* 将来如果你要为 Kubernetes 开发一个网络插件时，应该重点考虑的是如何配置这个 Pod 的 Network Namespace，而不是每一个用户容器如何使用你的网络配置，这是没有意义的。

* 创建一个Pod中两个容器（Ubuntu 和 Nginx），共享一个 Volume。

```bash
$ nano ubuntu_and_nginx_share_volume.yaml
```

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: ubuntu-and-nginx-share-volume
spec:
  restartPolicy: Never
  volumes:
  - name: shared-data
    hostPath:      
      path: /var/data
  containers:
  - name: nginx-container
    image: nginx
    volumeMounts:
    - name: shared-data
      mountPath: /usr/share/nginx/html
  - name: ubuntu-container
    image: ubuntu
    volumeMounts:
    - name: shared-data
      mountPath: /pod-data
    command: ["/bin/sh"]
    args: ["-c", "echo Hello from the ubuntu container > /pod-data/index.html"]
```

```bash
$ kubectl apply -f ubuntu_and_nginx_share_volume.yaml
```

## 容器设计模式

### 第一个最典型的例子是：解藕 WAR 包与 Web 服务器。

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: javaweb-2
spec:
  initContainers:
  - image: geektime/sample:v2
    name: war
    command: ["cp", "/sample.war", "/app"]
    volumeMounts:
    - mountPath: /app
      name: app-volume
  containers:
  - image: geektime/tomcat:7.0
    name: tomcat
    command: ["sh","-c","/root/apache-tomcat-7.0.42-v2/bin/start.sh"]
    volumeMounts:
    - mountPath: /root/apache-tomcat-7.0.42-v2/webapps
      name: app-volume
    ports:
    - containerPort: 8080
      hostPort: 8001 
  volumes:
  - name: app-volume
    emptyDir: {}
```

* 在 Pod 中，所有 Init Container 定义的容器，都会比 spec.containers 定义的用户容器先启动。Init Container 容器会按顺序逐一启动，而直到它们都启动并且退出了，用户容器才会启动。

* 实际上，这个所谓的“组合”操作，正是容器设计模式里**最常用的一种模式**，它的名字叫：**sidecar**。

### 第二个例子，容器的日志收集。

```txt
我现在有一个应用，需要不断地把日志文件输出到容器的 /var/log 目录中。
这时，我就可以把一个 Pod 里的 Volume 挂载到应用容器的 /var/log 目录上。
然后，我在这个 Pod 里同时运行一个 sidecar 容器，它也声明挂载同一个 Volume 到自己的 /var/log 目录上。
这样，接下来 sidecar 容器就只需要做一件事儿，那就是不断地从自己的 /var/log 目录里读取日志文件，转发到 MongoDB 或者 Elasticsearch 中存储起来。这样，一个最基本的日志收集工作就完成了。
```

* Istio 项目使用 sidecar 容器完成微服务治理。

### 总结

```txt
一个容器永远只能管理一个进程。更确切地说，一个容器，就是一个进程。
Pod 的本质：Pod，实际上是在扮演传统基础设施里“虚拟机”的角色；而容器，则是这个虚拟机里运行的用户程序。
当你需要把一个运行在虚拟机里的应用迁移到 Docker 容器中时，一定要仔细分析到底有哪些进程（组件）运行在这个虚拟机里。然后，你就可以把整个虚拟机想象成为一个 Pod，把这些进程分别做成容器镜像，把有顺序关系的容器，定义为 Init Container。这才是更加合理的、松耦合的容器编排诀窍，也是从传统应用架构，到“微服务架构”最自然的过渡方式。
```

### 参考资料
* [Borg、Omega和Kubernetes：谷歌十几年来从这三个容器管理系统中得到的经验教训](http://dockone.io/article/1153)
* [Design Patterns for Container-based Distributed Systems](https://www.usenix.org/conference/hotcloud16/workshop-program/presentation/burns)
* [基于容器的分布式系统的设计模式](http://dockone.io/article/1547)
* [基于容器的分布式系统的设计模式](https://zhuanlan.zhihu.com/p/34552109)
* [分布式系统设计介绍](https://zhuanlan.zhihu.com/p/34145849)
* [如何在 K8S 之上架构更高可用的应用](https://zhuanlan.zhihu.com/p/42884367)
* [K8s 容器设计模式实践，分散收集模式](https://www.infoq.cn/article/kubernetes-and-cloud-native-applications-part07)